<a href="https://colab.research.google.com/github/bernia/MachineLearning/blob/master/tf_tutorials/load_and_preprocess_data/CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load CSV data from a .csv to a DataFrame
%tensorflow_version 2.x 

from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [0]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [0]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [9]:
train_file_path
# head {train_file_path}

'/root/.keras/datasets/train.csv'

In [0]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [14]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'male']
age                 : [28. 28. 39. 56. 66.]
n_siblings_spouses  : [0 0 1 0 0]
parch               : [0 0 1 0 0]
fare                : [ 7.55   7.896 83.158 26.55  10.5  ]
class               : [b'Third' b'Third' b'First' b'First' b'Second']
deck                : [b'unknown' b'unknown' b'E' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'n' b'y' b'y']


In [15]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'female' b'female' b'male']
age                 : [31. 22. 35. 24.  3.]
n_siblings_spouses  : [1 0 0 0 4]
parch               : [1 0 0 0 2]
fare                : [26.25   9.    21.    69.3   31.388]
class               : [b'Second' b'Third' b'Second' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'B' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [17]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [41. 42. 30.  2. 11.]
n_siblings_spouses  : [0. 1. 1. 4. 5.]
parch               : [2. 0. 1. 1. 2.]
fare                : [20.212 52.    24.15  29.125 46.9  ]


In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [20]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 28.5     0.      0.      7.229]
 [ 21.      0.      0.     73.5  ]
 [ 30.      0.      0.     12.475]
 [ 38.      0.      0.    227.525]
 [ 28.      0.      0.      7.896]]

[0 0 1 1 0]


In [21]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'female']
age                 : [19. 24. 28. 28. 33.]
n_siblings_spouses  : [0 0 0 0 1]
parch               : [0 0 0 0 0]
fare                : [ 7.896 13.     7.75   6.858 53.1  ]
class               : [b'Third' b'Second' b'Third' b'Third' b'First']
deck                : [b'unknown' b'F' b'unknown' b'unknown' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Queenstown' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']


In [0]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [25]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']
numeric             : [[28.     0.     0.     7.225]
 [10.     3.     2.    27.9  ]
 [21.     0.     0.     7.796]
 [20.     0.     0.     9.846]
 [11.     5.     2.    46.9  ]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [27]:
# Data normalization

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


In [30]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f91628ab0d0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [31]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[70.5  ,  0.   ,  0.   ,  7.75 ],
       [48.   ,  1.   ,  0.   , 76.729],
       [27.   ,  0.   ,  0.   ,  7.925],
       [20.   ,  0.   ,  0.   ,  9.5  ],
       [28.   ,  0.   ,  0.   , 15.05 ]], dtype=float32)>

In [32]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 3.266, -0.474, -0.479, -0.488],
       [ 1.468,  0.395, -0.479,  0.776],
       [-0.21 , -0.474, -0.479, -0.485],
       [-0.77 , -0.474, -0.479, -0.456],
       [-0.13 , -0.474, -0.479, -0.354]], dtype=float32)

In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [35]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [36]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [38]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     1.     3.266 -0.474
 -0.479 -0.488  1.     0.   ]


In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [41]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 1s 12ms/step - loss: 0.4947 - accuracy: 0.7384
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4205 - accuracy: 0.8278
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4045 - accuracy: 0.8198
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3966 - accuracy: 0.8309
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3800 - accuracy: 0.8293
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3710 - accuracy: 0.8517
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3660 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3563 - accuracy: 0.8405
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3668 - accuracy: 0.8357
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3554 - accuracy: 0.853

In [42]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     53/Unknown - 0s 8ms/step - loss: 0.4425 - accuracy: 0.8447

Test Loss 0.44247437774572734, Test Accuracy 0.8446969985961914


In [43]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))



Predicted survival: -204.76%  | Actual outcome:  SURVIVED
Predicted survival: -113.83%  | Actual outcome:  DIED
Predicted survival: -219.12%  | Actual outcome:  SURVIVED
Predicted survival: 407.55%  | Actual outcome:  DIED
Predicted survival: -94.32%  | Actual outcome:  SURVIVED
